In [1]:
%%javascript
IPython.notebook.kernel.execute('nb_name = "' + IPython.notebook.notebook_name + '"')

<IPython.core.display.Javascript object>

In [2]:
%load_ext autoreload
%autoreload
from IPython.display import clear_output

import os
os.environ['PYTHONHASHSEED'] = '0'
os.environ["NUMEXPR_NUM_THREADS"] = "1"
os.environ["MKL_NUM_THREADS"] = "1"
os.environ["OMP_NUM_THREADS"] = "1"

import sys
import argparse
import subprocess
import pdb
import time
import random
import _pickle as cPickle
import matplotlib.pyplot as plt
import glob

%matplotlib inline

import numpy as np
import pandas as pd
import tensorflow as tf

from data_structure import get_batches
from hntm import HierarchicalNeuralTopicModel
from nhdp import nestedHierarchicalNeuralTopicModel
from tsgntm import TreeStructuredGaussianNeuralTopicModel
from tree import get_descendant_idxs
from evaluation import get_hierarchical_affinity, get_topic_specialization, print_topic_sample, print_topic_year
from configure import get_config

pd.set_option('display.max_columns', 30)

# load data & set config

In [3]:
config = get_config(nb_name)

In [4]:
os.environ["CUDA_VISIBLE_DEVICES"] = config.gpu
np.random.seed(config.seed)
random.seed(config.seed)

In [5]:
instances_train, instances_dev, instances_test, word_to_idx, idx_to_word, bow_idxs = cPickle.load(open(config.path_data,'rb'))
train_batches = get_batches(instances_train, config.batch_size)
dev_batches = get_batches(instances_dev, config.batch_size)
test_batches = get_batches(instances_test, config.batch_size)
config.dim_bow = len(bow_idxs)

In [6]:
def debug(variable, sample_batch=None):
    if sample_batch is None: sample_batch = test_batches[0][1]
    feed_dict = model.get_feed_dict(sample_batch, mode='test')
    _variable = sess.run(variable, feed_dict=feed_dict)
    return _variable

def check(variable):
    sess = tf.Session()
    sess.run(tf.global_variables_initializer())
    sample_batch = test_batches[0]
    feed_dict = model.get_feed_dict(sample_batch, mode='test', assertion=True)
    _variable = sess.run(variable, feed_dict=feed_dict)
    return _variable

# model

## initialize model

In [7]:
if 'sess' in globals(): sess.close()
TopicModels = {'hntm': HierarchicalNeuralTopicModel, 'nhdp': nestedHierarchicalNeuralTopicModel, 'tsgntm': TreeStructuredGaussianNeuralTopicModel}
TopicModel = TopicModels[config.model]
model = TopicModel(config)
sess = tf.Session(config=tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1))
sess.run(tf.global_variables_initializer())
saver = tf.train.Saver(max_to_keep=config.max_to_keep)
update_tree_flg = False

# run

## initialize log

In [8]:
checkpoint = []
losses_train = []
ppls_train = []
ppl_min = np.inf
epoch = 0
train_batches = get_batches(instances_train, config.batch_size, iterator=True)

log_df = pd.DataFrame(columns=pd.MultiIndex.from_tuples(
                    list(zip(*[['','','','TRAIN:','','','','','','VALID:','','','','','','TEST:','', 'SPEC:', '', '', 'HIER:', ''],
                            ['Time','Ep','Ct','LOSS','PPL','NLL','KL', 'GAUSS', 'REG','LOSS','PPL','NLL','KL', 'GAUSS','REG','LOSS','PPL', '1', '2', '3', 'CHILD', 'OTHER']]))))

cmd_rm = 'rm -r %s' % config.dir_model
res = subprocess.call(cmd_rm.split())
cmd_mk = 'mkdir %s' % config.dir_model
res = subprocess.call(cmd_mk.split())

def update_checkpoint(config, checkpoint, global_step):
    checkpoint.append(config.path_model + '-%i' % global_step)
    if len(checkpoint) > config.max_to_keep:
        path_model = checkpoint.pop(0) + '.*'
        for p in glob.glob(path_model):
            os.remove(p)
    cPickle.dump(checkpoint, open(config.path_checkpoint, 'wb'))
    
def validate(sess, batches, model):
    losses = []
    ppl_list = []
    prob_topic_list = []
    n_bow_list = []
    n_topics_list = []
    for ct, batch in batches:
        feed_dict = model.get_feed_dict(batch, mode='test')
        loss_batch, topic_loss_recon_batch, topic_loss_kl_batch, topic_loss_gauss_batch, topic_loss_reg_batch, ppls_batch, prob_topic_batch, n_bow_batch, n_topics_batch \
            = sess.run([model.loss, model.topic_loss_recon, model.topic_loss_kl, model.topic_loss_gauss, model.topic_loss_reg, model.topic_ppls, model.prob_topic, model.n_bow, model.n_topics], feed_dict = feed_dict)
        losses += [[loss_batch, topic_loss_recon_batch, topic_loss_kl_batch, topic_loss_gauss_batch, topic_loss_reg_batch]]
        ppl_list += list(ppls_batch)
        prob_topic_list.append(prob_topic_batch)
        n_bow_list.append(n_bow_batch)
        n_topics_list.append(n_topics_batch)
    loss_mean, topic_loss_recon_mean, topic_loss_kl_mean, topic_loss_gauss_mean, topic_loss_reg_mean = np.mean(losses, 0)
    ppl_mean = np.exp(np.mean(ppl_list))
    
    probs_topic = np.concatenate(prob_topic_list, 0)
    
    n_bow = np.concatenate(n_bow_list, 0)
    n_topics = np.concatenate(n_topics_list, 0)
    probs_topic_mean = np.sum(n_topics, 0) / np.sum(n_bow)
    
    return loss_mean, topic_loss_recon_mean, topic_loss_kl_mean, topic_loss_gauss_mean, topic_loss_reg_mean, ppl_mean, probs_topic_mean    

## train & validate model

In [9]:
time_start = time.time()
while epoch < config.n_epochs:
    # train
    for ct, batch in train_batches:
        feed_dict = model.get_feed_dict(batch)
        _, loss_batch, topic_loss_recon_batch, topic_loss_kl_batch, topic_loss_gauss_batch, topic_loss_reg_batch, ppls_batch, global_step_log = \
        sess.run([model.opt, model.loss, model.topic_loss_recon, model.topic_loss_kl, model.topic_loss_gauss, model.topic_loss_reg, model.topic_ppls, tf.train.get_global_step()], feed_dict = feed_dict)

        losses_train += [[loss_batch, topic_loss_recon_batch, topic_loss_kl_batch, topic_loss_gauss_batch, topic_loss_reg_batch]]
        ppls_train += list(ppls_batch)

#         if global_step_log % config.log_period == 0:
        if global_step_log % 1000 == 0:
            # validate
            loss_train, topic_loss_recon_train, topic_loss_kl_train, topic_loss_gauss_train, topic_loss_reg_train = np.mean(losses_train, 0)
            ppl_train = np.exp(np.mean(ppls_train))
            loss_dev, topic_loss_recon_dev, topic_loss_kl_dev, topic_loss_gauss_dev, topic_loss_reg_dev, ppl_dev, probs_topic_dev = validate(sess, dev_batches, model)

            # test
            if ppl_dev < ppl_min:
                ppl_min = ppl_dev
                loss_test, _, _, _, _, ppl_test, _ = validate(sess, test_batches, model)
                saver.save(sess, config.path_model, global_step=global_step_log)
                cPickle.dump(config, open(config.path_config % global_step_log, 'wb'))
                update_checkpoint(config, checkpoint, global_step_log)
            
            # visualize topic
            topics_freq_indices = np.argsort(sess.run(model.topic_bow), 1)[:, ::-1][:, :config.n_freq]
            topics_freq_idxs = bow_idxs[topics_freq_indices]
            topic_freq_tokens = {topic_idx: [idx_to_word[idx] for idx in topic_freq_idxs] for topic_idx, topic_freq_idxs in zip(model.topic_idxs, topics_freq_idxs)}
            topic_prob_topic = {topic_idx: prob_topic for topic_idx, prob_topic in zip(model.topic_idxs, probs_topic_dev)}
            descendant_idxs = {parent_idx: get_descendant_idxs(model, parent_idx) for parent_idx in model.topic_idxs}
            recur_prob_topic = {parent_idx: np.sum([topic_prob_topic[child_idx] for child_idx in recur_child_idxs]) for parent_idx, recur_child_idxs in descendant_idxs.items()}
            
            depth_specs = get_topic_specialization(sess, model, instances_test)
            hierarchical_affinities = get_hierarchical_affinity(sess, model)
            
            # log
            clear_output()
            time_log = int(time.time() - time_start)
            log_series = pd.Series([time_log, epoch, ct, \
                    '%.2f'%loss_train, '%.0f'%ppl_train, '%.2f'%topic_loss_recon_train, '%.2f'%topic_loss_kl_train, '%.2f'%topic_loss_gauss_train, '%.2f'%topic_loss_reg_train, \
                    '%.2f'%loss_dev, '%.0f'%ppl_dev, '%.2f'%topic_loss_recon_dev, '%.2f'%topic_loss_kl_dev, '%.2f'%topic_loss_gauss_dev, '%.2f'%topic_loss_reg_dev, \
                    '%.2f'%loss_test, '%.0f'%ppl_test, \
                    '%.2f'%depth_specs[1], '%.2f'%depth_specs[2], '%.2f'%depth_specs[3], \
                    '%.2f'%hierarchical_affinities[0], '%.2f'%hierarchical_affinities[1]],
                    index=log_df.columns)
            log_df.loc[global_step_log] = log_series
            display(log_df)
            cPickle.dump(log_df, open(os.path.join(config.path_log), 'wb'))
            print_topic_sample(sess, model, topic_prob_topic=topic_prob_topic, recur_prob_topic=recur_prob_topic, topic_freq_tokens=topic_freq_tokens)
            print(np.sum(debug(model.topic_logvars), 1))
            
            # update tree
            if not config.static:
                config.tree_idxs, update_tree_flg = model.update_tree(topic_prob_topic, recur_prob_topic)
                if update_tree_flg:
                    print(config.tree_idxs)
                    name_variables = {tensor.name: variable for tensor, variable in zip(tf.global_variables(), sess.run(tf.global_variables()))} # store paremeters
                    if 'sess' in globals(): sess.close()
                    model = HierarchicalNeuralTopicModel(config)
                    sess = tf.Session(config=tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1))
                    name_tensors = {tensor.name: tensor for tensor in tf.global_variables()}
                    sess.run([name_tensors[name].assign(variable) for name, variable in name_variables.items()]) # restore parameters
                    saver = tf.train.Saver(max_to_keep=1)
                
            time_start = time.time()

    train_batches = get_batches(instances_train, config.batch_size, iterator=True)
    epoch += 1

loss_dev, topic_loss_recon_dev, topic_loss_kl_dev, topic_loss_reg_dev, ppl_dev, probs_topic_dev = validate(sess, dev_batches, model)
topics_freq_indices = np.argsort(sess.run(model.topic_bow), 1)[:, ::-1][:, :config.n_freq]
topics_freq_idxs = bow_idxs[topics_freq_indices]
topic_freq_tokens = {topic_idx: [idx_to_word[idx] for idx in topic_freq_idxs] for topic_idx, topic_freq_idxs in zip(model.topic_idxs, topics_freq_idxs)}
topic_prob_topic = {topic_idx: prob_topic for topic_idx, prob_topic in zip(model.topic_idxs, probs_topic_dev)}
descendant_idxs = {parent_idx: get_descendant_idxs(model, parent_idx) for parent_idx in model.topic_idxs}
recur_prob_topic = {parent_idx: np.sum([topic_prob_topic[child_idx] for child_idx in recur_child_idxs]) for parent_idx, recur_child_idxs in descendant_idxs.items()}
display(log_df)
print_topic_sample(sess, model, topic_prob_topic=topic_prob_topic, recur_prob_topic=recur_prob_topic, topic_freq_tokens=topic_freq_tokens)

TRAIN:                                  VALID:        \
      Time  Ep    Ct    LOSS   PPL     NLL    KL GAUSS   REG    LOSS   PPL   
1000    61   0   999  929.08  2708  924.71  1.21  3.16  0.25  890.30  2597   
2000    50   0  1999  922.36  2556  917.09  1.99  3.28  0.24  880.71  2364   
3000    51   1   295  919.52  2477  913.80  2.42  3.30  0.24  878.20  2304   
4000    52   1  1295  916.76  2425  910.70  2.69  3.37  0.24  873.94  2218   
5000    52   1  2295  914.52  2387  908.26  2.85  3.41  0.23  873.04  2208   
6000    53   2   591  913.71  2361  907.30  2.97  3.44  0.23  874.29  2224   
7000    51   2  1591  912.61  2340  906.06  3.06  3.48  0.23  870.75  2156   
8000    52   2  2591  911.56  2323  904.89  3.15  3.53  0.23  870.12  2151   
9000    50   3   887  910.86  2307  904.05  3.23  3.59  0.23  871.73  2175   
10000   51   3  1887  910.17  2293  903.20  3.30  3.66  0.23  870.81  2157   
11000   52   4   183  909.58  2279  902.48  3.38  3.73  0.22  867.57  2089   
12000   50   4  1183  908.96  2267  901.71  3.45  3.81  0.22  868.33  2093   
13000   51   4  2183  908.31  2255  900.92  3.51  3.88  0.22  866.51  2059   
14000   50   5   479  907.92  2244  900.41  3.57  3.93  0.22  866.87  2069   
15000   51   5  1479  907.49  2234  899.87  3.63  3.99  0.22  865.99  2054   
16000   52   5  2479  906.93  2225  899.22  3.67  4.04  0.21  865.89  2050   
17000   51   6   775  906.56  2217  898.76  3.72  4.08  0.21  866.56  2063   
18000   51   6  1775  906.24  2210  898.35  3.76  4.13  0.21  867.98  2088   
19000   51   7    71  905.93  2203  897.98  3.79  4.16  0.21  866.93  2067   
20000   51   7  1071  905.54  2196  897.52  3.82  4.20  0.21  867.76  2086   
21000   51   7  2071  905.20  2190  897.12  3.85  4.23  0.21  866.08  2053   
22000   54   8   367  905.00  2184  896.86  3.88  4.26  0.21  864.14  2015   
23000   51   8  1367  904.74  2179  896.55  3.91  4.29  0.20  864.71  2019   
24000   51   8  2367  904.42  2174  896.18  3.93  4.31  0.20  864.77  2026   
25000   52   9   663  904.27  2170  895.97  3.95  4.34  0.20  866.31  2069   
26000   54   9  1663  904.05  2166  895.71  3.98  4.36  0.20  863.24  2002   
27000   52   9  2663  903.82  2161  895.44  4.00  4.38  0.20  863.81  2011   
28000   52  10   959  903.62  2158  895.20  4.02  4.40  0.20  866.12  2061   
29000   52  10  1959  903.42  2154  894.96  4.04  4.42  0.20  864.39  2022   
30000   52  11   255  903.31  2150  894.81  4.05  4.44  0.20  863.26  2004   
31000   52  11  1255  903.11  2147  894.58  4.07  4.46  0.20  863.65  2012   
32000   54  11  2255  902.90  2144  894.33  4.09  4.48  0.20  863.22  1996   

                                  TEST:       SPEC:             HIER:        
          NLL    KL GAUSS   REG    LOSS   PPL     1     2     3 CHILD OTHER  
1000   885.69  1.49  3.12  0.22  888.58  2549  0.16  0.16  0.14  0.95  0.92  
2000   874.73  2.95  3.03  0.30  879.79  2370  0.29  0.20  0.21  0.93  0.83  
3000   871.00  3.48  3.71  0.27  878.46  2321  0.18  0.22  0.25  0.93  0.82  
4000   866.65  3.75  3.54  0.28  876.26  2291  0.26  0.24  0.23  0.92  0.80  
5000   865.59  3.81  3.64  0.21  874.92  2263  0.20  0.23  0.23  0.92  0.82  
6000   866.97  3.64  3.68  0.15  874.92  2263  0.24  0.22  0.25  0.94  0.81  
7000   863.24  3.75  3.77  0.27  872.67  2225  0.25  0.22  0.24  0.92  0.78  
8000   862.39  3.93  3.80  0.19  871.15  2189  0.21  0.20  0.27  0.90  0.81  
9000   863.62  4.04  4.07  0.25  871.15  2189  0.27  0.20  0.26  0.89  0.76  
10000  862.39  4.17  4.25  0.17  871.15  2189  0.20  0.24  0.25  0.91  0.75  
11000  858.81  4.27  4.49  0.15  870.63  2169  0.19  0.23  0.27  0.89  0.76  
12000  859.41  4.29  4.63  0.23  870.63  2169  0.19  0.21  0.28  0.87  0.75  
13000  857.43  4.39  4.69  0.19  867.20  2105  0.36  0.23  0.28  0.89  0.71  
14000  857.65  4.46  4.77  0.16  867.20  2105  0.24  0.21  0.28  0.85  0.73  
15000  856.70  4.48  4.80  0.21  867.81  2111  0.22  0.22  0.26  0.90  0.72  
16000  856.61  4.53  4.75  0.16  865.50  20

0 R: 1.000 P: 0.012 music kids room show hotel cool wings enjoyed games enjoy
   1 R: 0.365 P: 0.065 fried soup beef meat flavor portions decent dishes huge fish
     11 R: 0.137 P: 0.137 rice thai mexican spicy tacos chinese rolls soup beef noodles
     12 R: 0.059 P: 0.059 years found fried decent fantastic beef meat full huge wo
     13 R: 0.105 P: 0.105 enjoyed wonderful fantastic steak husband bread large wine meat absolutely
   2 R: 0.204 P: 0.033 coffee sandwich ice free cream store stop inside options drive
     21 R: 0.022 P: 0.022 coffee sandwich cream ice sandwiches bread options today french drive
     22 R: 0.094 P: 0.094 coffee ice cream tea shop store chocolate free flavors line
     23 R: 0.055 P: 0.055 sandwich bacon sandwiches eggs brunch burgers bread options french toast
   3 R: 0.175 P: 0.026 wings music cool check decent waitress enjoyed patio enjoy game
     31 R: 0.022 P: 0.022 wings wine attentive fantastic waitress manager enjoyed check music wonderful
     32

KeyboardInterrupt: 

In [11]:
debug(model.topic_means)[1]

array([ 2.8216894 , -3.0563464 ,  1.6908246 , -1.1562389 , -2.7861776 ,
       -0.16849104, -2.979567  ,  2.3647585 , -0.24711254,  2.3434775 ,
        1.2681077 , -2.037859  , -0.5537858 ,  3.1777048 , -2.3174355 ,
        0.81516606,  1.3252438 ,  0.9859205 , -3.2249868 ,  0.00921591,
       -1.2574761 ,  2.7307491 ,  2.9201877 , -2.4252245 ,  0.7486757 ,
        2.6120846 , -0.5086255 , -2.6265562 , -1.9473915 , -2.147787  ,
       -0.47880453, -2.537198  ], dtype=float32)

In [15]:
debug(model.topic_means)[model.topic_idxs.index(model.config.tree_idxs[1][0])]

array([ 3.4633958e+00, -3.3375292e+00,  2.9749236e+00, -2.5223880e+00,
       -3.1319203e+00,  4.4205226e-04, -3.4287007e+00,  2.8137615e+00,
       -2.7193124e+00,  3.1666677e+00,  3.2885613e+00, -2.4302793e+00,
       -1.6794885e+00,  3.0809908e+00, -3.2836900e+00,  2.2672522e+00,
        1.5040021e+00,  1.7132781e+00, -3.0915594e+00, -2.8817942e+00,
       -3.3942351e+00,  3.0325408e+00,  3.0610740e+00, -3.4285488e+00,
       -2.4458025e+00,  2.6225791e+00, -2.7167537e+00, -2.6694596e+00,
       -2.8730149e+00, -3.1766460e+00, -2.1321492e+00, -3.5230451e+00],
      dtype=float32)

# analysis

In [21]:
def get_topic_year(sample_batches):
    probs_topics = []
    years = []
    for i, sample_batch in sample_batches:
        probs_topics_batch = sess.run(model.prob_topic, feed_dict=model.get_feed_dict(sample_batch, mode='test'))
        years_batch = [instance.year for instance in sample_batch]
        probs_topics += [probs_topics_batch]
        years += years_batch
    probs_topics = np.concatenate(probs_topics)
    years = np.array(years)

    topic_years = years.dot(probs_topics) / np.sum(probs_topics, 0)
    topic_year = {model.topic_idxs[i]: year for i, year in enumerate(topic_years)}
    return topic_year

In [22]:
sample_batches = get_batches(instances_train, config.batch_size)
topic_year = get_topic_year(sample_batches)
print_topic_year(sess, model, topic_freq_tokens=topic_freq_tokens, topic_year=topic_year)

0 Avg Year: 2005 level analysis english systems domain lexical linguistic test process lexicon
   1 Avg Year: 2000 grammar structure rules semantic form representation rule lexical type structures
     11 Avg Year: 2006 verb semantic syntactic verbs event relations discourse argument relation arguments
     12 Avg Year: 2007 tree parsing dependency parser grammar trees node parse rules syntactic
   2 Avg Year: 2008 english languages dictionary errors morphological chinese lexical rules corpora pos
     21 Avg Year: 2009 entity sense relations relation semantic entities wordnet senses lexical patterns
     22 Avg Year: 2010 translation english source alignment phrase target languages sentences systems parallel
   3 Avg Year: 2008 models probability training algorithm probabilities search parameters sequence segmentation gram
     31 Avg Year: 2011 features feature training performance learning classifier classification accuracy test class
     32 Avg Year: 2015 models network vector emb